In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from iminuit import Minuit
from scipy import stats
import math 
import sympy as sp
from IPython.core.display import Latex
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [42]:
sys.path.append('External_Functions')
from ExternalFunctions import Chi2Regression, nice_string_output
from ExternalFunctions import add_text_to_ax 

In [87]:
class Error_Prop:
    def __init__(self, eq, sym_var, sym_err, vals, err_vals, corr):
        self.eq = eq
        self.sym_var = sym_var
        self.sym_err = sym_err
        self.vals = vals
        self.corr = corr
        self.covar = self.corr.copy()
        for i in range(len(self.covar)):
            for j in range(len(self.covar)):
                self.covar[i][j] *= err_vals[i] * err_vals[j]
        self.derivatives = [self.eq.diff(var) for var in sym_var]
        self.functions = [sp.lambdify((*sym_var, *sym_err), derive) for derive in self.derivatives]
        self.diff_vals = [func(*vals, *err_vals) for func in self.functions]

    def get_propEq(self):
        propEq = 0
        for i in range(len(self.diff_vals)):
            for j in range(len(self.diff_vals)):
                propEq += self.derivatives[i] * self.derivatives[j] * self.corr[i][j]* self.sym_err[i] * self.sym_err[j]
        return sp.sqrt(propEq)

    def get_error(self):
        error = 0
        for i in range(len(self.diff_vals)):
            for j in range(len(self.diff_vals)):
                error += self.diff_vals[i] * self.diff_vals[j] * self.covar[i][j]
        return np.sqrt(error)
    
    def get_contributions(self):
        contributions = []
        for i in range(len(self.diff_vals)):
            contributions.append(self.diff_vals[i]**2 * self.covar[i][i])
        return contributions
    


In [88]:
x, y, z = sp.symbols('x y z')
dx, dy, dz = sp.symbols('sigma_x sigma_y sigma_z')

vx = 1.92 
vdx = 0.39

vy = 3.1 
vdy = 1.3

vz = 4.2
vdz = 1.2

z1 = x*y + z**2 

sym = np.array([x, y, z])
sig_sym = np.array([dx, dy, dz])
vals = np.array([vx, vy, vz])
sig_vals = np.array([vdx, vdy, vdz])
corr = np.array([[1, 0.5, 0], [0.5, 1, 0], [0, 0, 1]])

propagator = Error_Prop(z1, sym, sig_sym, vals, sig_vals, corr)
display(propagator.get_propEq())
print(propagator.get_error())
print(propagator.get_contributions())


2.0*sqrt(0.25*sigma_x**2*y**2 + 0.25*sigma_x*sigma_y*x*y + 0.25*sigma_y**2*x**2 + sigma_z**2*z**2)

10.597913049275315
[1.4616810000000002, 6.230016000000001, 101.6064]


In [ ]:
def norm_gauss(x, mu, sigma):
    return stats.norm.pdf(x, mu, sigma)

def gauss(x, mu, sigma, N):
    return N*norm_gauss(x, mu, sigma)

def double_gauss(x, mu_1, sigma_1, mu_2, sigma_2, N, f):
    return N*(f*norm_gauss(x, mu_1, sigma_1) + (1-f)*norm_gauss(x, mu_2, sigma_2))



In [ ]:
def linear(x, a, b):
    return a*x + b

def poly2(x, a, b, c):
    return a*x**2 + b*x + c

def poly3(x, a, b, c, d):
    return a*x**3 + b*x**2 + c*x + d
    

In [ ]:
def create_histogram(data, n_bins):
    counts, bin_edges = np.histogram(data, bins=n_bins, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2
    bin_width = bin_edges[1]-bin_edges[0]
    return counts, bin_edges, bin_centers, bin_width


In [ ]:
def accept_reject(N_points, xrange, yrange, random, func, *args):
    N_tries = 0
    accepted = []
    for i in range(N_points):
        while True:
            N_tries += 1
            x_test = random.uniform(*xrange)
            y_test = random.uniform(*yrange)
            if y_test < func(x_test, *args):
                break
        accepted.append(x_test)
    return accepted, N_tries

In [90]:
#Accept reject combined method

def accept_reject_combi(N_points, random, func, test_func, inv_func, *args):
    N_tries = 0
    accepted = []
    for i in range(N_points):
        while True:
            N_tries += 1
            x_test = inv_func(random.uniform(size = 1))
            y_test = random.uniform(0, test_func(x_test))
            if y_test < func(x_test, *args):
                break
        accepted.append(x_test)
    return accepted, N_tries

In [ ]:
def evaluate_chi2(minuit_obj, len_counts):
    chi2 = minuit_obj.fval
    ndof = len_counts - len(minuit_obj.values[:])
    chi2_prob = stats.chi2.sf(chi2, ndof)
    return chi2, ndof, chi2_prob    

In [ ]:
#Fisher method
def fisher(data_1, data_2, w_0 = 0):
    from numpy.linalg import inv
    mu_1 = np.mean(data_1, axis=1)
    mu_2 = np.mean(data_2, axis=1)
    cov_sum = np.cov(data_1, rowvar=False) + np.cov(data_2, rowvar=False)
    w = inv(cov_sum) @ (mu_1 - mu_2)
    return w @ data_1 + w_0, w @ data_2 + w_0



In [ ]:
#Decision tree?

In [ ]:
#Roc curve


In [ ]:
#Penalty for chi2